In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
from nltk.corpus import stopwords
import re
import numpy as np

In [2]:
df = pd.read_csv('train.csv', index_col='pair_id')
df.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0


In [3]:
duplicates = df[df['is_duplicate']==1]
no_duplicates = df[df['is_duplicate']==0]

In [4]:
duplicates.shape

(3658, 3)

245251

In [5]:
def split_dataframe(df, chunk_size = 10000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [6]:
df_split = split_dataframe(no_duplicates, 245251)

In [7]:
tobe_duplicates = df_split[0]
frames = [df_split[1], df_split[2]]
no_duplicates = pd.concat(frames)

In [8]:
tobe_duplicates_second_name = tobe_duplicates.drop(['name_2', 'is_duplicate'], axis=1)
tobe_duplicates_second_name.head()

,name_1
pair_id,
1,Iko Industries Ltd.
2,Apcotex Industries Ltd.
3,"Rishichem Distributors Pvt., Ltd."
4,Powermax Rubber Factory
5,Tress A/S


In [9]:
list_name_2 = tobe_duplicates_second_name.values.tolist()

In [10]:
list_name_2

[['Iko Industries Ltd.'],
 ['Apcotex Industries Ltd.'],
 ['Rishichem Distributors Pvt., Ltd.'],
 ['Powermax Rubber Factory'],
 ['Tress A/S'],
 ['Wpt Mexico Sa De Cv'],
 ['National Bank Of'],
 ['Citibank Korea Inc.'],
 ['Prettl Electric Del Bajio S.A. De C.V.'],
 ['Beijing Zhongyi Rongda Tech Trading Co., Ltd.'],
 ['R.I.Intl'],
 ['Angle Industries Ltd.'],
 ['Bestocean Worldwide Logistics Inc.'],
 ['Uni Trans Illinois Consolidated Inc.'],
 ['Warehouse Jonesboro'],
 ['Ktn Underwood Arlanxeo Usa Llc'],
 ['Wynn Express Corp'],
 ['Parker Industrials De Rl De Cv'],
 ['Air Sea Transport (Chicago) Inc.'],
 ['Shanghai Scientific Instruments And Materials Co., Ltd.'],
 ['Faj International'],
 ['Galata Chemicals Llc'],
 ['Battery Technology Inc.'],
 ['Ksb Shipping & Logistics Llc'],
 ['H.R. International'],
 ['S.R. International'],
 ['D S International'],
 ['Huizhou City Sanhe Industrial Co., Ltd.'],
 ['Palziv North America'],
 ['Salnik Solutions'],
 ['Qingdao Dongyong Changshi Logistics Co., Ltd.

In [11]:
list_name_2_new = []

In [12]:
stop_words = ['ООО', 'ОАО', 'ЗАО', 'ИП','Ltd','Co','LLC','LLP', 'Inc', 'Corp.', 'LDC', 'IBC', 'SA', 'GmbH', 'SARL', 'BV', 'NV', 'AVV', 'AG', 'IC', '& Со','LP','.',',', '"' , 'Ltd.','Co.','LLC.','LLP.', 'Inc.', 'Corp', 'LDC.', 'IBC.', 'SA.', 'GmbH.', 'SARL.', 'Industries', 'BV.', 'NV.', 'AVV.', 'AG.', 'IC.', '& Со.','LP.', 'Llc', 'GROUP', 'Corporation', 'Corporate']

In [13]:
def text_filter(text):
    global company_name
    clear_tokens = []
    tokens = word_tokenize(text)
    for token in tokens:
        if token not in stop_words:
            clear_tokens.append(token)
            company_name = (' '.join(clear_tokens))#.lower()#.replace(" ", "")
            company_name = re.sub("\(.*?\)","", company_name)
    return company_name

In [14]:
for i in list_name_2:
    for company in i:
        company_new = text_filter(company)
        list_name_2_new.append(company_new)
            

In [15]:
tobe_duplicates['name_2']= list_name_2_new
tobe_duplicates.insert(2, "is_duplicate", 1)
tobe_duplicates.tail(5)

ValueError: cannot insert is_duplicate, already exists

In [ ]:
frames = [duplicates, tobe_duplicates]
duplicates_new = pd.concat(frames)

In [ ]:
frames = [duplicates_new, no_duplicates]
companies = pd.concat(frames).sort_index()

In [ ]:
companies.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,Iko,1
2,Apcotex Industries Ltd.,Apcotex,1
3,"Rishichem Distributors Pvt., Ltd.",Rishichem Distributors Pvt.,1
4,Powermax Rubber Factory,Powermax Rubber Factory,1
5,Tress A/S,Tress A/S,1


In [152]:
companies = pd.read_csv('companies.csv', index_col='pair_id')
companies = companies[['is_duplicate','name_1','name_2']].drop(['is_duplicate'],axis=1)
# duplicates_1000 = pd.read_csv('duplicates_1000.csv', index_col='pair_id')
# duplicates_1000 = companies[['is_duplicate','name_1','name_2']].drop(['is_duplicate'],axis=1)

In [153]:
companies_list = companies.values.tolist()

In [157]:
companies_list_dict = list(set(companies['name_1'].values.tolist()))

In [159]:
len(companies_list_dict)

17656

In [19]:
import difflib
def similarity(s1, s2):
  normalized1 = s1.lower()
  normalized2 = s2.lower()
  matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
  return matcher.ratio()

In [175]:
def finder(text):
    counter = 0
    answer = []
    for company in companies_list_dict:
        company.lower()
        tokens = word_tokenize(company)
        for token in tokens:
            shozhest = similarity(text, token)
            if counter < 5:
            # if text.lower() in company.lower():
                # print('Возможно вы имели в виду: ', i[0], '\nСовпадение: ', shozhest*100,'%')
                counter += 1
                answer.append([company, shozhest])
                if counter == 5:
                    answer.sort(key=lambda x: x[1], reverse=True)
            elif shozhest > answer[-1][1]:
                for answer_i in range(len(answer)):
                    if answer[answer_i][1] < shozhest:
                        answer.insert(answer_i, [company, shozhest])
                        answer.pop(-1)
                        break
    return answer

In [176]:
# def finder_1(text):
#     counter = 0
#     answer = []
#     for company in companies_list_dict:
#         company.lower()
#         tokens = word_tokenize(company)
#         shozhest = 0
#         for token in tokens:
#             shozhest += similarity(text, token)
#         shozhest /= len(tokens)
#         if counter < 5:
#         # if text.lower() in company.lower():
#             # print('Возможно вы имели в виду: ', i[0], '\nСовпадение: ', shozhest*100,'%')
#             counter += 1
#             answer.append([company, shozhest])
#             if counter == 5:
#                 answer.sort(key=lambda x: x[1], reverse=True)
#         elif shozhest > answer[-1][1]:
#             for answer_i in range(len(answer)):
#                 if answer[answer_i][1] < shozhest:
#                     answer.insert(answer_i, [company, shozhest])
#                     answer.pop(-1)
#                     break
#     return answer

In [222]:
def finder_2(text):
    text_tokens = word_tokenize(text)
    counter = 0
    answer = []
    for company in companies_list_dict:
        company.lower()
        tokens = word_tokenize(company)
        shozhest = 0
        len_shozhest = 0
        len_tokens = len(tokens)
        for text_token in text_tokens:
            for token_i, token in enumerate(tokens):
                shozhest += similarity(text_token, token)                 
        # shozhest = (len_tokens + len(text_tokens))
        if counter < 5:
        # if text.lower() in company.lower():
            # print('Возможно вы имели в виду: ', i[0], '\nСовпадение: ', shozhest*100,'%')
            counter += 1
            answer.append([company, shozhest])
            if counter == 5:
                answer.sort(key=lambda x: x[1], reverse=True)
        elif shozhest > answer[-1][1]:
            for answer_i in range(len(answer)):
                if answer[answer_i][1] < shozhest:
                    answer.insert(answer_i, [company, shozhest])
                    answer.pop(-1)
                    break
    return answer

In [178]:
def finder_3(text):
    counter = 0
    answer = []
    for company in companies_list_dict:
        company.lower()
        shozhest = similarity(text, company)
        if counter < 5:
        # if text.lower() in company.lower():
            # print('Возможно вы имели в виду: ', i[0], '\nСовпадение: ', shozhest*100,'%')
            counter += 1
            answer.append([company, shozhest])
            if counter == 5:
                answer.sort(key=lambda x: x[1], reverse=True)
        elif shozhest > answer[-1][1]:
            for answer_i in range(len(answer)):
                if answer[answer_i][1] < shozhest:
                    answer.insert(answer_i, [company, shozhest])
                    answer.pop(-1)
                    break
    return answer

In [231]:
from tqdm import tqdm

In [241]:
finder('JX Nippon Oil')

[['Nippon Express De Mexico S.A.De C.V', 0.631578947368421],
 ['JX Nippon Oil & Gas Exploration (Myanmar) Limited', 0.631578947368421],
 ['Nippon Leakless Talbros Pvt., Ltd.', 0.631578947368421],
 ['Nippon Express Group', 0.631578947368421],
 ['Nippon Signal Co., Ltd. (TSE:6741)', 0.631578947368421]]

In [242]:
def metrika_finder(path_dataset):
    dataset = pd.read_csv(path_dataset)
    accuracy_top_1 = 0
    accuracy_top_5 = 0
    len_dataset = 100
    for i in tqdm(range(len_dataset)):
        finded = finder(dataset['name_2'][i]) 
        if dataset['is_duplicate'][i]==1 and finded[0][0] == dataset['name_1'][i]:
            accuracy_top_1 += 1
        finded = finded[:][0]
        if dataset['is_duplicate'][i]==1 and dataset['name_1'][i] in finded:
            accuracy_top_5 += 1

    accuracy_top_1 /= len_dataset
    accuracy_top_5 /= len_dataset
    return accuracy_top_1, accuracy_top_5
        

In [228]:
x = 'Predtol' #Ryohin Keikaku Reliance India Private Ltd. - finder
y = 'Iko ' #Ravago Shah Polymers Pvt., Ltd.
z = 'Prettl Electric Del Bajio' #Sumitomo Industrias Pesadas Do Brasil Ltda -finder2, finder3
main_x = y

In [243]:
metrika_finder('duplicates_1000.csv')

100%|██████████| 100/100 [05:44<00:00,  3.45s/it]


(0.03, 0.03)

In [239]:
dp = pd.read_csv('duplicates_1000.csv')
dp.head(10)

,pair_id,name_1,name_2,is_duplicate
0,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,1
1,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",1
2,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,1
3,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,1
4,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),1
5,1787,Pt Bridgestone Tire Indonesia,Bridgestone Do Brasil Industria E Comenrcio Ltda,1
6,1799,Brenntag Peru S.A.C.,Brenntag Group,1
7,1963,"Carlisle Coatings & Waterproofing, Inc.",Carlisle Coatings & Wtrprfng,1
8,2055,Pt Bridgestone Tire Indonesia,"Thai Bridgestone Co., Ltd.",1
9,2773,Onduline Polska Sp. z o.o.,Onduline S.A.,1


In [225]:
finder(main_x)

[['Shanghai Light Industries Products', 0.8],
 ['Srf Industries (Thailand) Ltd.', 0.8],
 ['Pt Lotus Indah Textile Industries', 0.8],
 ['Kapoor Industries', 0.8],
 ['Black Rose Industries Ltd.', 0.8]]

In [226]:
finder_2(main_x)

[['Adams Resources Exploration Corporation, Oil And Gas Assets In Crocket And Irion Counties In Texas',
  6.887934812006019],
 ['MOL Magyar Olaj- es Gazipari Nyilvanosan Mukodo Reszvenytarsasag (BUSE:MOL) - MOL Magyar Olaj és Gázipari Nyilvánosan Működő Részvénytársaság',
  6.265651015651016],
 ['Qing Yuan Shi Qing Cheng Qu Shi Jiao Gao Dun Pi Ge Shu Zhi Pin Chang',
  6.0641025641025665],
 ['Qingdao Free Trade Zone Logistic Zone Vonder International Logistic Co.',
  5.516042688485144],
 ['Section Modulus Hot Thinking (Suzhou) Industrial Electric Equipment Co., Ltd.',
  5.158239997017088]]

In [229]:
finder_3(main_x)

[['IKO', 0.8571428571428571],
 ['iko sales', 0.6153846153846154],
 ['Iko Europe', 0.5714285714285714],
 ['Sika Sa', 0.5454545454545454],
 ['Iko Calgary', 0.5333333333333333]]

In [72]:
y = 'Bridgestone Neumaticos'
x = 'Bridgestone Neumaticos Company'
tokens_x = word_tokenize(x)
tokens_y = word_tokenize(y)

In [73]:
tokens_x

['Bridgestone', 'Neumaticos', 'Company']

In [20]:
def find(text):
    for i in companies_list:
        for company in i:
            company.lower()
            shozhest = similarity(text, company)
            if text.lower() in company.lower():
                # print('Возможно вы имели в виду: ', i[0], '\nСовпадение: ', shozhest*100,'%')
                return i[0]

In [200]:
companies.head(10)

,name_1,name_2
pair_id,,
1,Iko Industries Ltd.,Iko
2,Apcotex Industries Ltd.,Apcotex
3,"Rishichem Distributors Pvt., Ltd.",Rishichem Distributors Pvt.
4,Powermax Rubber Factory,Powermax Rubber Factory
5,Tress A/S,Tress A/S
6,Wpt Mexico Sa De Cv,Wpt Mexico Sa De Cv
7,National Bank Of,National Bank Of
8,Citibank Korea Inc.,Citibank Korea
9,Prettl Electric Del Bajio S.A. De C.V.,Prettl Electric Del Bajio S.A. De C.V


In [ ]:
from tqdm import tqdm

In [230]:
import torch

In [ ]:
device = torch.device('cuda:0' 
                      if torch.cuda.is_available() 
                      else 'cpu')

In [ ]:
def metriki(path_dataset):
    dataset = pd.read_csv(path_dataset)
    accuracy = 0
    for i in tqdm(range(len(dataset))):
        finded = find(dataset['name_2'][i])
        if dataset['is_duplicate']==1 and finded == dataset['name_1'][i]:
            accuracy += 1
    accuracy /= len(dataset)
    return accuracy
        
    

In [ ]:
metriki('companies_4000.csv')

 53%|█████▎    | 2102/4000 [01:37<04:38,  6.81it/s]

In [ ]:
companies.shape

(497819, 3)

In [ ]:
torch.device('cuda:0')

device(type='cuda', index=0)

In [ ]:
duplicates = pd.read_csv('duplicates.csv', index_col='pair_id')

In [ ]:
df_split = split_dataframe(duplicates, 1000)
duplicates_1000 = df_split[0]

In [ ]:
duplicates_1000.to_csv('duplicates_1000.csv')

In [ ]:
no_duplicates = pd.read_csv('no_duplicates.csv', index_col='pair_id')

In [ ]:
df_split = split_dataframe(no_duplicates, 2000)
no_duplicates_2000 = df_split[0]

In [ ]:
xui = [duplicates_2000, no_duplicates_2000]
companies_new = pd.concat(xui).sort_index()

In [ ]:
companies_new.to_csv('companies_4000.csv')